In [15]:
!pip install facenet-pytorch
!pip install torch torchvision
!pip install mtcnn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import torch
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image, UnidentifiedImageError
import cv2

# Load the pre-trained FaceNet model
model = InceptionResnetV1(pretrained='vggface2').eval()

# Initialize MTCNN for face detection
mtcnn = MTCNN(image_size=160, margin=0)

In [17]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    face = mtcnn(image)
    if face is None:
        raise ValueError("No face detected in the image.")
    return face

In [18]:
import os
database_dir = '../data/lfw_funneled'

# os.listdir(database_dir)

In [19]:
database = {}

for person_name in os.listdir(database_dir):
    person_dir = os.path.join(database_dir, person_name)
    if os.path.isdir(person_dir):
        embeddings = []
        for file_name in os.listdir(person_dir):
            if file_name.endswith(('jpg', 'jpeg', 'png')):
                image_path = os.path.join(person_dir, file_name)
                try:
                    face = preprocess_image(image_path)
                    face = face.unsqueeze(0)  # Add batch dimension
                    embedding = model(face).detach()  # Generate embedding
                    embeddings.append(embedding)
                except ValueError as e:
                    print(f"Skipping {file_name} in {person_name}: {e}")
        if embeddings:
            database[person_name] = torch.stack(embeddings).mean(dim=0)

print(f"Database size: {len(database)}")


Skipping Andre_Agassi_0012.jpg in Andre_Agassi: No face detected in the image.
Skipping Andy_Roddick_0009.jpg in Andy_Roddick: No face detected in the image.
Skipping Angelina_Jolie_0018.jpg in Angelina_Jolie: No face detected in the image.
Skipping Ariel_Sharon_0067.jpg in Ariel_Sharon: No face detected in the image.
Skipping Arnold_Schwarzenegger_0004.jpg in Arnold_Schwarzenegger: No face detected in the image.
Skipping Arnold_Schwarzenegger_0017.jpg in Arnold_Schwarzenegger: No face detected in the image.
Skipping Atal_Bihari_Vajpayee_0002.jpg in Atal_Bihari_Vajpayee: No face detected in the image.
Skipping Atal_Bihari_Vajpayee_0010.jpg in Atal_Bihari_Vajpayee: No face detected in the image.
Skipping Bill_Clinton_0012.jpg in Bill_Clinton: No face detected in the image.
Skipping Bill_Clinton_0024.jpg in Bill_Clinton: No face detected in the image.
Skipping Bill_Gates_0008.jpg in Bill_Gates: No face detected in the image.
Skipping Bill_Simon_0003.jpg in Bill_Simon: No face detected in

In [20]:
def euclidean_distance(embedding1, embedding2):
    return torch.dist(embedding1, embedding2).item()

def find_closest_match(embedding, database, threshold=1.1):
    closest_distance = float('inf')
    closest_name = None
    for name, db_embedding in database.items():
        distance = euclidean_distance(embedding, db_embedding)
        if distance < closest_distance:
            closest_distance = distance
            closest_name = name
    if closest_distance < threshold:
        return closest_name, closest_distance
    else:
        return None, closest_distance

In [21]:
# Path to the new image to be matched
new_image_path = 'yo.jpg'

# Preprocess the new image
new_face = preprocess_image(new_image_path)
new_face = new_face.unsqueeze(0)  # Add batch dimension
new_embedding = model(new_face).detach()  # Generate embedding

# Find the closest match in the database
name, distance = find_closest_match(new_embedding, database)
if name:
    print(f"Match found: {name} with distance {distance}")
else:
    print("No match found within the threshold.")

Match found: 01-Jose with distance 0.0
